In [ ]:
!pip install --upgrade azureml-core

In [ ]:
!pip install --upgrade scikit-learn==0.24.2

In [ ]:
!pip install --upgrade imbalanced-learn==0.8.0

In [ ]:
!pip install --upgrade seaborn

In [ ]:
!pip install --upgrade azureml-interpret

In [ ]:
!pip install interpret-community[visualization]

In [ ]:
!pip install --upgrade azureml-mlflow

In [ ]:
import azureml.core


print(f"Azure ML SDK Version: {azureml.core.VERSION}")

In [ ]:
from azureml.core import Workspace


ws = Workspace(subscription_id="23376daf-77f3-4195-9649-e223b8072ad9",
              resource_group="rg-handsonaml",
              workspace_name="wshandsonaml"
              )
ws

https://docs.microsoft.com/fr-fr/azure/machine-learning/how-to-setup-authentication#use-service-principal-authentication

In [ ]:
import sklearn


print(f"Azure ML SDK Version: {sklearn.__version__}")

In [ ]:
experiment_name = "exp_remote_training"

In [ ]:
from azureml.core import Experiment


exp = Experiment(workspace=ws, name=experiment_name)
exp

In [ ]:
compute_target_name = "compute-cluster"

In [ ]:
# Compute target creation

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=compute_target_name)
    print(" Cluster already exists")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',
                                                           min_nodes=0, max_nodes=2)
    cpu_cluster = ComputeTarget.create(ws, compute_target_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True, min_node_count=0, timeout_in_minutes=30)

In [ ]:
# Retrieve existing compute target

from azureml.core.compute import ComputeTarget


compute_target = ComputeTarget(workspace=ws, name=compute_target_name)
print(compute_target.provisioning_state)

In [ ]:
from azureml.core import Datastore, Dataset


datastore_name = 'workspaceblobstore'
  
# retrieve an existing datastore in the workspace by name
datastore = Datastore.get(ws, datastore_name)

# create a TabularDataset from path(s) in datastore
datastore_paths = [(datastore, 'handson/wine.csv')]

ds = Dataset.Tabular.from_delimited_files(path=datastore_paths)

In [ ]:
ds.register(workspace=ws,
            name='ds_wine_by_code',
            description='Wine registered dataset'
           )

# Remote training

In [ ]:
script_folder = 'scripts'

import os

os.makedirs(script_folder, exist_ok=True)

In [ ]:
%%writefile scripts/train.py

import argparse
import os
import sys
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix,  plot_roc_curve, f1_score, recall_score
import joblib

import mlflow
import mlflow.sklearn

from azureml.core import Run
from azureml.core import Dataset

# get hold of the current run
run = Run.get_context()
exp = run.experiment
ws = run.experiment.workspace

parser = argparse.ArgumentParser()
parser.add_argument('--penalty', type=str, default='l2', help='norm')
parser.add_argument('--max_iter', type=int, default=10000, help='iterations')
args = parser.parse_args()

print("Argument 1: %s" % args.penalty)
print("Argument 2: %s" % args.max_iter)

dataset_name = "ds_wine_by_code"
dataset = Dataset.get_by_name(ws, name=dataset_name)
df = dataset.to_pandas_dataframe()

label = "class"
X = np.array(df.drop(label, axis=1))
y = np.array(df[label])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = LogisticRegression(penalty=args.penalty, max_iter=args.max_iter).fit(X=X_train, y=y_train)

run.log('train-accuracy', model.score(X_train, Y_train))
run.log('test-accurary', model.score(X_test, Y_test))

run.log('recall', recall_score(model.predict(X_test), Y_test))
run.log('f1', f1_score(model.predict(X_test), Y_test))

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/wine_classification_model.pkl')

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies


trainenv = Environment('wine-training-env')
trainenv.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'joblib',
    'numpy',
    'pandas',
    'sklearn',
    'mlflow',
    'matplotlib',
    'seaborn'
])

# https://azure.github.io/azureml-cheatsheets/docs/cheatsheets/python/v1/environment/
# add pip packages
#conda.add_pip_package('pickle')

trainenv.save_to_directory('environment/trainenv.yml', overwrite=True)



Conda channels : https://docs.anaconda.com/anaconda/user-guide/tasks/using-repositories/


In [ ]:
%%writefile scripts/trainenv.yml

name: trainenv
channels:
    - anaconda
    - conda-forge
dependencies:
    - python=3.6.9
    - pip
    - pip:
        - azureml-core
        - azureml-defaults
        - azureml-mlflow
        - opencensus-ext-azure>=1.0.1
        - inference-schema[numpy-support]
        - joblib
        - numpy
        - pandas
        - scikit-learn==0.24.1
        - imbalanced-learn==0.8.0
        - mlflow
        - matplotlib
        - seaborn

In [ ]:
from azureml.core import Environment


trainenv = Environment.from_conda_specification('trainenv', 'scripts/trainenv.yml')

In [ ]:
#DEPRECATED
from azureml.train.estimator import Estimator


script_params = {
    '--penalty': 'l2',
    '--max_iter': 10000
}

estimator = Estimator(source_directory=script_folder,
              script_params=script_params,
              compute_target=compute_target_name,
              environment_definition=trainenv,
              entry_script='train.py')

#run = exp.submit(config=estimator)

ScriptRunConfig

https://docs.microsoft.com/fr-fr/python/api/azureml-core/azureml.core.scriptrunconfig?view=azure-ml-py

https://docs.microsoft.com/fr-fr/azure/machine-learning/how-to-migrate-from-estimators-to-scriptrunconfig

In [ ]:
from azureml.core import ScriptRunConfig


config = ScriptRunConfig(source_directory=script_folder,
                        script='train.py',
                        arguments=['--penalty', 'l2', '--max_iter', 10000],
                        compute_target=compute_target_name,
                        environment=trainenv)

In [ ]:
run = exp.submit(config=config)
run

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

In [ ]:
# specify show_output to True for a verbose log
run.wait_for_completion(show_output=True)

Warning: you have pip-installed dependencies in your environment file, but you do not list pip itself as one of your conda dependencies.  Conda may not use the correct pip to install your packages, and they may end up in the wrong place.  Please add an explicit pip dependency.  I'm adding one for you, but still nagging you.

==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base -c defaults conda

In [ ]:
print(run.get_metrics())

In [ ]:
print(run.get_file_names())

In [ ]:
aml_model = run.register_model(model_name='wine_classification_model', model_path='outputs/wine_classification_model.pkl')

# Inference

In [ ]:
%%writefile scripts/score.py

import joblib
import numpy as np
import os
import json
import sklearn

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType


# The init() method is called once, when the web service starts up.
#
# Typically you would deserialize the model file, as shown here using joblib,
# and store it in a global variable so your run() method can access it later.
def init():
    global model

    # The AZUREML_MODEL_DIR environment variable indicates
    # a directory containing the model file you registered.
    model_filename = 'german_credit_log_model.pkl'
    model_path = os.path.join(os.environ['AZUREML_MODEL_DIR'], model_filename)
    #ModuleNotFoundError: No module named 'sklearn.externals.joblib'
    model = joblib.load(model_path)


# The run() method is called each time a request is made to the scoring API.
#
# Shown here are the optional input_schema and output_schema decorators
# from the inference-schema pip package. Using these decorators on your
# run() method parses and validates the incoming payload against
# the example input you provide here. This will also generate a Swagger
# API document for your web service.
#@input_schema('data', NumpyParameterType(np.array([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])))
#@output_schema(NumpyParameterType(np.array([0])))

def run(raw_data):
    data = json.loads(raw_data)['data']
    method = json.loads(raw_data)['method']
    # Use the model object loaded by init().
    result = model.predict(data) if method=="predict" else model.predict_proba(data)

    # You can return any JSON-serializable object.
    return result.tolist()

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies


environment = Environment('german-credit-deploy-env')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'joblib',
    'numpy',
    'sklearn'
])

environment.save_to_directory('environment/infenv.yml', overwrite=True)

In [ ]:
%%writefile scripts/infenv.yml

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-defaults
  - inference-schema[numpy-support]
  - joblib
  - numpy
  - sklearn
channels:
- anaconda
- conda-forge


In [ ]:
from azureml.core.model import InferenceConfig


inference_config = InferenceConfig(entry_script='score.py', environment=environment)

In [ ]:
service_name = 'wine-custom-srv'

In [ ]:
from azureml.core import Webservice
from azureml.core.webservice import AciWebservice
from azureml.exceptions import WebserviceException
from azureml.core.model import Model

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=False)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[aml_model],
                       inference_config=inference_config,
                       deployment_config=aci_config)

service.wait_for_deployment(show_output=True)

In [ ]:
print(service.state)

In [ ]:
print(service.get_logs())

In [ ]:
# Seulement si auth_enabled=True
#print(service.get_keys())

In [ ]:
print(service.swagger_uri)

In [ ]:
import json


input_payload = json.dumps({ 
    "data": [
        [14.23,1.71,2.43,15.6,127,2.8,3.06,.28,2.29,5.64,1.04,3.92,1065]
    ],
    "method": "predict"  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)


In [ ]:
import json


input_payload = json.dumps({ 
    "data": [
        [14.23,1.71,2.43,15.6,127,2.8,3.06,.28,2.29,5.64,1.04,3.92,1065]
    ],
    "method": "predict_proba"  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)


In [ ]:
import requests


input_data = "{\"data\": [[14.23,1.71,2.43,15.6,127,2.8,3.06,.28,2.29,5.64,1.04,3.92,1065]], \"method\":\"predict\"}"

headers = {'Content-Type':'application/json'}

# for AKS deployment you'd need to the service key in the header as well
#api_key = service.get_keys()[0]
#headers = {'Content-Type':'application/json',  'Authorization':('Bearer '+ api_key)} 

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
print("prediction:", resp.text)

In [ ]:
#service.delete()